In [1]:
import pandas as pd
"""
file reading resulted in an error. encoding iso fixed the issue reading unkown character
"""
df = pd.read_csv("master.csv", encoding = "ISO-8859-1")

df['Operating Expenses FY'] = df['Operating Expenses FY'].str.replace(r'[^0-9]', '', regex=True).astype(float)
df['Operating Expenses FY'] = pd.to_numeric(df['Operating Expenses FY'], errors='coerce')
df['Operating Expenses FY'] = df['Operating Expenses FY'] / 100

df['Unlinked Passenger Trips FY'] = df['Unlinked Passenger Trips FY'].str.replace(r'[^0-9]', '', regex=True).astype(float)
df['Unlinked Passenger Trips FY'] = pd.to_numeric(df['Unlinked Passenger Trips FY'], errors='coerce')

df['Avg Cost Per Trip FY'] = df['Avg Cost Per Trip FY'].str.replace(r'[^0-9]', '', regex=True).astype(float)
df['Avg Cost Per Trip FY'] = pd.to_numeric(df['Avg Cost Per Trip FY'], errors='coerce')
df['Avg Cost Per Trip FY'] = df['Avg Cost Per Trip FY'] / 100
df['Revenue'] = (df['Avg Cost Per Trip FY'] * df['Unlinked Passenger Trips FY']) - df['Operating Expenses FY']

df['Passenger Miles FY'] = df['Passenger Miles FY'].str.replace(r'[^0-9]', '', regex=True)
df['Passenger Miles FY'] = pd.to_numeric(df['Passenger Miles FY'], errors='coerce')
df = df[df['Passenger Miles FY'] != 0]

df = df[~(df['Status'].isna() | (df['Status'] == 'Inactive'))]

print(df['Status'])
#print(df['Revenue'])



0       Active
2       Active
3       Active
5       Active
6       Active
         ...  
2282    Active
2283    Active
2284    Active
2285    Active
2286    Active
Name: Status, Length: 1288, dtype: object
